In [2]:
import sys, os 
sys.path.insert(0, os.path.dirname(os.getcwd()))

from wofscast.data_utils import extract_input_target_times
import xarray as xr 

In [3]:
path = '/work/mflora/wofs-cast-data/dataset.nc'
dataset = xr.load_dataset(path)

In [18]:
dataset

<xarray.Dataset>
Dimensions:              (time: 73, Time: 1, south_north: 300, west_east: 300,
                          bottom_top: 50, bottom_top_stag: 51,
                          soil_layers_stag: 9, west_east_stag: 301,
                          south_north_stag: 301)
Coordinates:
    XLAT                 (Time, south_north, west_east) float32 31.02 ... 39.21
    XLONG                (Time, south_north, west_east) float32 -85.68 ... -7...
  * time                 (time) timedelta64[ns] 00:00:00 00:05:00 ... 06:00:00
Dimensions without coordinates: Time, south_north, west_east, bottom_top,
                                bottom_top_stag, soil_layers_stag,
                                west_east_stag, south_north_stag
Data variables: (12/75)
    COMPOSITE_REFL_10CM  (time, Time, south_north, west_east) float32 0.0 ......
    COSALPHA             (time, Time, south_north, west_east) float32 0.998 ....
    COSZEN               (time, Time, south_north, west_east) float32 0.06262...
    GRAUP_ACC_NC         (time, Time, south_north, west_east) float32 0.0 ......
    HAILCAST_DIAM_MAX    (time, Time, south_north, west_east) float32 0.0 ......
    HAIL_MAX2D           (time, Time, south_north, west_east) float32 0.0 ......
    ...                   ...
    VT_DBZ_WT            (time, Time, bottom_top, south_north, west_east) float32 ...
    W                    (time, Time, bottom_top_stag, south_north, west_east) float32 ...
    WSPD10MAX            (time, Time, south_north, west_east) float32 5.521 ....
    WSPD80               (time, Time, south_north, west_east) float32 -99.0 ....
    W_UP_MAX             (time, Time, south_north, west_east) float32 0.2716 ...
    XLAND                (time, Time, south_north, west_east) float32 1.0 ......
Attributes: (12/74)
    TITLE:                            OUTPUT FROM WRF V3.9pre#2 MODEL
    START_DATE:                      2020-05-06_00:00:00
    WEST-EAST_GRID_DIMENSION:        301
    SOUTH-NORTH_GRID_DIMENSION:      301
    BOTTOM-TOP_GRID_DIMENSION:       51
    DX:                              3000.0
    ...                              ...
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      -1
    ETAC:                            0.0
    history:                         Fri Jun 12 01:27:58 2020: /scratch/softw...
    NCO:                             4.6.7

In [14]:
# Formating the time dimension for the graphcast code. 
import pandas as pd

# Define the start time for your dataset
start_time = pd.Timestamp('2020-05-06 00:00:00')

# Assuming `ds` is your xarray.Dataset
num_time_points = dataset.sizes['time']

# Determine the time resolution (e.g., '1H' for hourly, '1D' for daily)
time_resolution = '5min'  # Example: 1-hour resolution

# Generate the datetime range
time_range = pd.date_range(start=start_time, periods=num_time_points, freq=time_resolution)

dataset['time'] = time_range

# Convert 'time' dimension to timedeltas from the first time point
time_deltas = (dataset['time'] - dataset['time'][0]).astype('timedelta64[ns]')
dataset['time'] = time_deltas

In [16]:
input_duration = '15min'
target_lead_times = '1 hour'

input_ds, target_ds = extract_input_target_times(dataset, input_duration, target_lead_times)

In [17]:
input_ds

<xarray.Dataset>
Dimensions:              (time: 3, Time: 1, south_north: 300, west_east: 300,
                          bottom_top: 50, bottom_top_stag: 51,
                          soil_layers_stag: 9, west_east_stag: 301,
                          south_north_stag: 301)
Coordinates:
    XLAT                 (Time, south_north, west_east) float32 31.02 ... 39.21
    XLONG                (Time, south_north, west_east) float32 -85.68 ... -7...
  * time                 (time) timedelta64[ns] -1 days +23:50:00 ... 00:00:00
Dimensions without coordinates: Time, south_north, west_east, bottom_top,
                                bottom_top_stag, soil_layers_stag,
                                west_east_stag, south_north_stag
Data variables: (12/75)
    COMPOSITE_REFL_10CM  (time, Time, south_north, west_east) float32 0.0 ......
    COSALPHA             (time, Time, south_north, west_east) float32 0.998 ....
    COSZEN               (time, Time, south_north, west_east) float32 -0.6541...
    GRAUP_ACC_NC         (time, Time, south_north, west_east) float32 0.0 ......
    HAILCAST_DIAM_MAX    (time, Time, south_north, west_east) float32 0.0 ......
    HAIL_MAX2D           (time, Time, south_north, west_east) float32 0.0 ......
    ...                   ...
    VT_DBZ_WT            (time, Time, bottom_top, south_north, west_east) float32 ...
    W                    (time, Time, bottom_top_stag, south_north, west_east) float32 ...
    WSPD10MAX            (time, Time, south_north, west_east) float32 4.516 ....
    WSPD80               (time, Time, south_north, west_east) float32 8.504 ....
    W_UP_MAX             (time, Time, south_north, west_east) float32 0.4098 ...
    XLAND                (time, Time, south_north, west_east) float32 1.0 ......
Attributes: (12/74)
    TITLE:                            OUTPUT FROM WRF V3.9pre#2 MODEL
    START_DATE:                      2020-05-06_00:00:00
    WEST-EAST_GRID_DIMENSION:        301
    SOUTH-NORTH_GRID_DIMENSION:      301
    BOTTOM-TOP_GRID_DIMENSION:       51
    DX:                              3000.0
    ...                              ...
    ISURBAN:                         13
    ISOILWATER:                      14
    HYBRID_OPT:                      -1
    ETAC:                            0.0
    history:                         Fri Jun 12 01:27:58 2020: /scratch/softw...
    NCO:                             4.6.7